In [1]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
360869,7944051,3505619,1
443047,9283622,7831323,1
521573,10198699,3693255,1
335043,7523666,4064001,1
497657,9928586,3711313,1
...,...,...,...
82943,2214684,5205629,1
126829,4295860,3591928,1
332705,7486671,3711758,1
14407,1981717,3504080,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [4]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [5]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [6]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [7]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")
        


        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])
        
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [8]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [9]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
7/7 [==============================] - 3s 215ms/step - auc: 0.0000e+00 - rmse: 0.9991 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0200 - factorized_top_k/top_5_categorical_accuracy: 0.0637 - factorized_top_k/top_10_categorical_accuracy: 0.0938 - factorized_top_k/top_50_categorical_accuracy: 0.2087 - factorized_top_k/top_100_categorical_accuracy: 0.3013 - loss: 493.4719 - regularization_loss: 0.0000e+00 - total_loss: 493.4719
Epoch 2/3
7/7 [==============================] - 1s 189ms/step - auc: 0.0000e+00 - rmse: 0.9776 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.3175 - factorized_top_k/top_5_categorical_accuracy: 0.4850 - factorized_top_k/top_10_categorical_accuracy: 0.5763 - factorized_top_k/top_50_categorical_accuracy: 0.7738 - factorized_top_k/top_100_categorical_accuracy: 0.8625 - loss: 491.4693 - regularization_loss: 0.0000e+00 - total_loss: 491.4693
Epoch 3/3
7/7 [=======

In [10]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

7/7 [==============================] - 2s 207ms/step - auc: 0.0000e+00 - rmse: 0.9248 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9787 - factorized_top_k/top_5_categorical_accuracy: 0.9900 - factorized_top_k/top_10_categorical_accuracy: 0.9975 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 487.7170 - regularization_loss: 0.0000e+00 - total_loss: 487.7170


([0.0,
  0.9248433113098145,
  0.0,
  0.0,
  0.9787499904632568,
  0.9900000095367432,
  0.9975000023841858,
  1.0,
  1.0,
  109.46224975585938,
  0,
  109.46224975585938],
 [0.0, 0.0, 0.0])

In [11]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

4/4 [==============================] - 1s 225ms/step - auc: 0.0000e+00 - rmse: 0.9816 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.2350 - factorized_top_k/top_5_categorical_accuracy: 0.3550 - factorized_top_k/top_10_categorical_accuracy: 0.4550 - factorized_top_k/top_50_categorical_accuracy: 0.6600 - factorized_top_k/top_100_categorical_accuracy: 0.7600 - loss: 166.3080 - regularization_loss: 0.0000e+00 - total_loss: 166.3080


([0.0,
  0.9815613031387329,
  0.0,
  0.0,
  0.23499999940395355,
  0.35499998927116394,
  0.45500001311302185,
  0.6600000262260437,
  0.7599999904632568,
  16.63599395751953,
  0,
  16.63599395751953],
 [0.0, 0.0, 0.0])

In [12]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		0.9815613031387329
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		0.23499999940395355
factorized_top_k/top_5_categorical_accuracy		0.35499998927116394
factorized_top_k/top_10_categorical_accuracy		0.45500001311302185
factorized_top_k/top_50_categorical_accuracy		0.6600000262260437
factorized_top_k/top_100_categorical_accuracy		0.7599999904632568
AUC: 0.0


In [13]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 2022289,  9569103,  9936392,  9328336,  5834128,  9201428,
        5489439, 10661151,  8601626,  8168716,  7743304,  7774990,
        6063201,  9492293, 10130457, 11046709,  5250008,  6414436,
        7290087,  5177145,  2207123,  2008984,  5162544,  2082126,
        7987454,  9067346, 10850328, 11069974,  5628903,  9196312,
        8382816,  4752359,  5946316,  9309317,  4500203,  2237572,
        1951426,  8608437,  9930390,  5947056,  2161382,  9509453,
        9593564,  7929973,  7747793,  1943873, 10957378,  6958591,
        5773571,  2105130,  6067632,  6823962,  5463362,  2531606,
       10177777,  2173902,  4934629, 10669922,  2028249,  2002569,
        4142792,  5678466,  5911963, 10811468])

In [14]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  2022289
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
123,3473970,4062862948995,Reiterstiefel Glattleder braun,braun,51.617.22,6590677,315584,w,PT,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,42.0,TR,0.0,0.0,"[3473970, 3473971, 3473972, 3473974, 3473973, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
30,3473739,4062862951841,Sneaker low Materialmix Leder schwarz,schwarz,53.480.27,6590677,315567,w,VN,H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,EVA,0.0,0.0,"[3473739, 3473736, 3473732, 3473734, 3473731, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
32,548623,4058395881823,Sportliche Ballerina Rauleder schwarz,schwarz,02.643.47,6590678,315571,w,PT,F-S,...,bis 3 cm,46.0,11.0,20.0,0.0,Gummi,0.0,0.0,"[548623, 3482467, 3482470, 3482471, 3482472, 3...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
316,3743084,4062862105275,Sneaker low Materialmix Leder/Lederimitat grün,grün,56.966.35,4997827,315567,w,VN,H-W,...,3 cm - 5 cm,40.0,6.5,11.0,0.0,Gummi-EVA,0.0,0.0,"[3743084, 3743082, 3743086, 3743081, 3743085, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
328,3849864,4062862439554,Schnürstiefelette Leder Reptiloptik schwarz,schwarz,51.801.37,6590677,5014852,w,SK,H-W,...,3 cm - 5 cm,43.0,9.0,16.0,16.0,EVA,0.0,1.0,"[3849864, 3849865, 3849860, 3849854, 3849857, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
605,4770015,4062862777274,Eleganter Ballerina Rauleder blau,blau,61.300.16,6590677,315574,w,PT,F-S,...,bis 3 cm,40.0,6.5,11.0,0.0,TPU,0.0,0.0,"[4770015, 4770016, 4770010, 4770012, 4770017, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
647,5337911,4062862593898,Riemchensandale Glattleder weiß,weiß,64.671.21,6590677,315563,w,SK,F-S,...,3 cm - 5 cm,40.0,6.5,11.0,0.0,TR,0.0,0.0,"[5337911, 4945855, 4769002, 4945846, 4945847, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
731,4846598,4064032052839,Riemchensandale Glattleder beige,beige,62.761.59,6590678,315563,w,PT,F-S,...,bis 3 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0,"[4846598, 4846596, 4846602, 5150032, 5150034, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1273,6110096,4064032995433,Sportlicher Stiefel Glattleder schwarz,schwarz,71.629.27,6590677,5014850,w,PT,H-W,...,3 cm - 5 cm,43.0,9.0,16.0,41.0,EVA,1.0,1.0,"[6110096, 6509825, 6114875, 6114868, 6114871, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1543,6496831,4064032751732,Schnürstiefelette Rauleder Grau,Grau,73.734.19,6590677,315587,w,SK,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,17.0,TR,0.0,0.0,"[6496831, 7023180, 7023179, 8500093]","[38.0, 42.0, 43.0, 44.0]"
2567,9182571,4065171606785,Plateau Pumps Glattleder rosa,rosa,91.280.20,6590677,315569,w,PT,H-W,...,5 cm - 8 cm,42.0,8.0,14.0,0.0,TR,0.0,0.0,"[9182571, 9032029, 9032028, 9182566, 9182567, ...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 42...."


scores:  [0.02652754 0.02634018 0.02551009 0.02534236 0.02226981 0.02165984
 0.02137865 0.02120614 0.02074541 0.02049074]
